<table>
  <tr>
    <td><img src="https://github.com/rvss-australia/RVSS/blob/main/Pics/RVSS-logo-col.med.jpg?raw=1" width="400"></td>
    <td><div align="left"><font size="30">Color images</font></div></td>
  </tr>
</table>

(c) Peter Corke 2024

Robotics, Vision & Control: Python, see Chapter 11

## Configuring the Jupyter environment
We need to import some packages to help us with linear algebra (`numpy`), graphics (`matplotlib`), and machine vision (`machinevisiontoolbox`).
If you're running locally you need to have these packages installed.  If you're running on CoLab we have to first install machinevisiontoolbox which is not preinstalled, this will be a bit slow.

In [ ]:
try:
    import google.colab
    print('Running on CoLab')
    !pip install machinevision-toolbox-python
    COLAB = True
except:
    COLAB = False

%matplotlib widget
import matplotlib.pyplot as plt

import numpy as np
from machinevisiontoolbox import *
import ipywidgets as widgets

# display result of assignments
if COLAB:
    %config ZMQInteractiveShell.ast_node_interactivity = 'last_expr_or_assign'
# make NumPy display a bit nicer
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:10.4g}" if abs(x) > 1e-10 else f"{0:10.4g}"})

# Color planes
We will read a color image into a Python object

In [ ]:
image = Image.Read("flowers4.png")
# image = Image.Read("flowers8.png")

The size of the image is

In [ ]:
image.size

that is, it is 640x426.

We can pull out the NumPy array that holds the pixels

In [ ]:
array = image.image
array

which is simply a big 3-dimensional array of 8-bit integers.  We can consider this as a stack of 2-dimensional tables each of shape (426,640) and we refer to these as color planes. The planes correspond to the colors red, green and blue, and the elements vary between 0 (zero amount of the corresponding color) and 255 (maximum amount of the corresponding color).

To make this clearer, we can access the value of the pixel at image coordinate (516,351), remember that's (horizontal, vertical) coordinate

In [ ]:
image[516,351]

which is a 3-element 1-dimensional array with `uint8` values.  This is the intensity of red, green and blue respectively.



**Q: what color is this pixel?**


<p style="border:3px; background-color:#FF0000; font-weight: bold; padding: 1em; text-align: center;">Internally the color images are stored in OpenCV's default color order which is blue, green, red.  This is the opposite of almost everything else in the world which works with a color order of red, green and blue (RGB).  The Toolbox hides OpenCV idiosyncrasies.</p>

We can see a lot of pertinant information about the image by

In [ ]:
print(image)

which says that the image contains 3 "color planes" named R, G and B, and each plane is 640x426.

And, we can display it as an image

In [ ]:
image.disp();

As we saw earlier, the image view is interactive. As you move the cursor over the image, the pixel coordinates and value are updated at the bottom of the window.  The displayed pixel values are always in the range 0 to 255 which are minimum and maximum possible values for the `uint8` pixel data type.

A toolbar provides some extra functionality.  You can select a region to get an expanded view, pan that selected region around, change the zoom level, or revert to the original view.

**Q: move the cursor over the image and see the pixel values at the bottom right.  Use the buttons with the square in it to select a region of interest (click the button, then click and drag a rectangle in the image window, the home button restores the original view). How do the pixel values vary as you explore different colored flowers?  (there are some very small blue flowers)**

# Color planes

We can also think of the color image as a stack of three greyscale images that each represent the amoung of redness, greenness and blueness (effectively what the scene looks like through a red, green or blue filter).

Note that we can have images with 2, 3, 4 or more planes.  Hyperspectral cameras can "see" uptp 20 colors (often called bands, short for spectral bands).  The number of planes in the image is given by

In [ ]:
image.nplanes

Some images might have 3 planes but they represent hue, saturation and intensity, rather than red, green and blue. The names of the planes, and which "layer" in the 3D-array they are in is given by 

In [ ]:
image.colororder


Let's let more closely at one of these color planes -- the red plane.

In [ ]:
red = image.red()
red

and we see that is a greyscale image.  The intensity of each pixel is the amount of red at the corresponding pixel in the original color image.

In [ ]:
red.disp()

**Q: why are red flowers so dark in this image?**

**Q: why are the white and yellow flowers so bright in this image?**

**Q: display the green and blue planes**

We could also have written `image.plane(0)`, `image.plane("R")`, or `image[0]`.

# Greyscale and color conversion

We can convert a color image to a greyscale image

In [ ]:
image.mono().disp()

where the greyscale value is a weighted average of the red, greend and blue values.  The hightest weighting is for green and the lowest weighting is for blue, to match the response of the human eye.

We could also colorize the red plane image from above. Here we say that the red value is equal to the corresponding greyscale value, while green and blue are set to zero.  The result is a color image, but the only color is red.

In [ ]:
red.colorize([1,0,0]).disp()

# Histograms

Often we are interested to know the distribution of the pixel values in each plane

In [ ]:
image.stats()

which shows that the pixels span the full range from 0 to 255.  A histogram provides more nuanced information

In [ ]:
hist = image.hist()
hist

In [ ]:
plt.figure()
hist.plot(type="frequency", style="stack")

We see that the blue channel has a large number of 0 values. We also see that all channels are overexposed, a large number of pixels have the maximum value of 255.

**Q: where are these overexposed pixels in the image?**

**Q: where are all the pixels with zero amounts of blue?**